In [1]:
import json

In [6]:
original = json.load(open("data/verses_clauses_dict.json", "r", encoding="utf-8"))
results = [json.loads(line) for line in open("data/all_parsed_clauses_gpt4o-mini.jsonl", encoding="utf-8").readlines()]

In [7]:
a = results[0]
a

{'verse': 'Parse this verse: ויהי כאשרתמו כלהגוי לעבור אתהירדן פ ויאמר יהוה אליהושע לאמר',
 'parsed_clauses': '["ויהי כאשר תמו כל הגוי לעבור את הירדן פ","ויאמר יהוה אל יהושע לאמר"]'}

In [8]:
def extract_result(line):
    return line["verse"].split(":")[1].strip(), line["parsed_clauses"].replace("```", "").replace("json", "").replace("\n", "")

In [9]:
extract_result(a)

('ויהי כאשרתמו כלהגוי לעבור אתהירדן פ ויאמר יהוה אליהושע לאמר',
 '["ויהי כאשר תמו כל הגוי לעבור את הירדן פ","ויאמר יהוה אל יהושע לאמר"]')

In [10]:
def extract_results(results):
    verse_clauses = {}
    for result in results:
        verse, clauses = extract_result(result)
        verse_clauses[verse] = clauses
    return verse_clauses

In [11]:
verse_clauses_results = extract_results(results)

In [12]:
verse_clauses_results

{'ויהי כאשרתמו כלהגוי לעבור אתהירדן פ ויאמר יהוה אליהושע לאמר': '["ויהי כאשר תמו כל הגוי לעבור את הירדן פ","ויאמר יהוה אל יהושע לאמר"]',
 'וראה אתה הכהן והנה נהפך שער לבן בבהרת ומראה עמק מןהעור צרעת הוא במכוה פרחה וטמא אתו הכהן נגע צרעת הוא': '["ראה אתה הכהן","והנה נהפך שער לבן בבהרת","ומראה עמק מןהעור","צרעת הוא במכוה פרחה","וטמא אתו הכהן","נגע צרעת הוא"]',
 'כיאיך אעלה אלאבי והנער איננו אתי פן אראה ברע אשר ימצא אתאבי': '["כיאיך אעלה אלאבי","והנער איננו אתי","פן אראה ברע אשר ימצא אתאבי"]',
 'ויאמר אמציהו לאיש האלהים ומהלעשות למאת הככר אשר נתתי לגדוד ישראל ויאמר איש האלהים יש ליהוה לתת לך הרבה מזה': '["ויאמר אמציהו לאיש האלהים","ומהלעשות למאת הככר אשר נתתי לגדוד ישראל","ויאמר איש האלהים יש ליהוה לתת לך הרבה מזה"]',
 'וישכימו אנשי העיר בבקר והנה נתץ מזבח הבעל והאשרה אשרעליו כרתה ואת הפר השני העלה עלהמזבח הבנוי': '["וישכימו אנשי העיר בבקר","והנה נתץ מזבח הבעל","והאשרה אשרעליו כרתה","ואת הפר השני העלה עלהמזבח הבנוי"]',
 'וכתבתם עלמזוזות ביתך ובשעריך': '["כתבתם עלמזוזות ביתך","ובשעריך"]',


In [13]:
nb_correct = 0
nb_total = 0

for verse, clauses in verse_clauses_results.items():
    nb_total += 1
    true_clauses = original[verse]["clauses"]
    clauses = json.loads(clauses)
    complexity = original[verse]["complexity"]
    print("res", clauses)
    print("cor", true_clauses, complexity)
    print("")
    if clauses == true_clauses:
        nb_correct += 1

res ['ויהי כאשר תמו כל הגוי לעבור את הירדן פ', 'ויאמר יהוה אל יהושע לאמר']
cor ['ויהי', 'כאשרתמו כלהגוי', 'לעבור אתהירדן פ', 'ויאמר יהוה אליהושע', 'לאמר'] simple

res ['ראה אתה הכהן', 'והנה נהפך שער לבן בבהרת', 'ומראה עמק מןהעור', 'צרעת הוא במכוה פרחה', 'וטמא אתו הכהן', 'נגע צרעת הוא']
cor ['וראה אתה הכהן', 'והנה', 'נהפך שער לבן בבהרת', 'ומראה עמק מןהעור', 'צרעת הוא', 'במכוה פרחה', 'וטמא אתו הכהן', 'נגע צרעת הוא'] simple

res ['כיאיך אעלה אלאבי', 'והנער איננו אתי', 'פן אראה ברע אשר ימצא אתאבי']
cor ['כיאיך אעלה אלאבי', 'והנער', 'איננו אתי', 'פן אראה ברע', 'אשר ימצא אתאבי'] simple

res ['ויאמר אמציהו לאיש האלהים', 'ומהלעשות למאת הככר אשר נתתי לגדוד ישראל', 'ויאמר איש האלהים יש ליהוה לתת לך הרבה מזה']
cor ['ויאמר אמציהו לאיש האלהים', 'ומה', 'לעשות למאת הככר', 'אשר נתתי לגדוד ישראל', 'ויאמר איש האלהים', 'יש ליהוה', 'לתת לך הרבה מזה'] simple

res ['וישכימו אנשי העיר בבקר', 'והנה נתץ מזבח הבעל', 'והאשרה אשרעליו כרתה', 'ואת הפר השני העלה עלהמזבח הבנוי']
cor ['וישכימו אנשי העיר בבקר', 'והנה',

JSONDecodeError: Expecting ',' delimiter: line 1 column 39 (char 38)

In [17]:
print(f"Correct parsing: {nb_correct} over {nb_total}.")
print(f"Percentage of correct parsing: {nb_correct / nb_total * 100} %")

Correct parsing: 207 over 1162.
Percentage of correct parsing: 17.814113597246127 %
